In [1]:
# https://developer.spotify.com/console/get-audio-features-several-tracks/?ids=4JpKVNYnVcJ8tuMKjAj50A,2NRANZE9UCmPAS5XVbXL40,24JygzOLM0EmRQeGtFcIcG

#https://www.kaggle.com/aeryan/spotify-music-analysis

#https://www.kaggle.com/cihanoklap/top-songs-on-spotify-what-makes-them-popular

In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "Charts CSV/regional-us-daily-latest.csv"
spotify_data_us = pd.read_csv(path)
spotify_data_us.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Stay (with Justin Bieber),The Kid LAROI,1863631,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
2,2,good 4 u,Olivia Rodrigo,1619876,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
3,3,Kiss Me More (feat. SZA),Doja Cat,1154660,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz
4,4,Ain't Shit,Doja Cat,997997,https://open.spotify.com/track/5lAnYvAIkSDNXqfo7DyFUm


In [4]:
spotify_data_us.to_csv("clean_us.csv", header=False, index=False)


In [5]:
path_clean = "clean_global.csv"
spotify_data_us_clean = pd.read_csv(path_clean)
spotify_data_us_clean.head()

,Position,Track Name,Artist,Streams,URL
0,1,Stay (with Justin Bieber),The Kid LAROI,1863631,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
1,2,good 4 u,Olivia Rodrigo,1619876,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
2,3,Kiss Me More (feat. SZA),Doja Cat,1154660,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz
3,4,Ain't Shit,Doja Cat,997997,https://open.spotify.com/track/5lAnYvAIkSDNXqfo7DyFUm
4,5,Beggin',Måneskin,980955,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e


In [6]:
spotify_data_us_clean["ID"] = spotify_data_us_clean["URL"].str[31:]
spotify_data_us_clean.head()

,Position,Track Name,Artist,Streams,URL,ID
0,1,Stay (with Justin Bieber),The Kid LAROI,1863631,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20,5HCyWlXZPP0y6Gqq8TgA20
1,2,good 4 u,Olivia Rodrigo,1619876,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG,4ZtFanR9U6ndgddUvNcjcG
2,3,Kiss Me More (feat. SZA),Doja Cat,1154660,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz,3DarAbFujv6eYNliUTyqtz
3,4,Ain't Shit,Doja Cat,997997,https://open.spotify.com/track/5lAnYvAIkSDNXqfo7DyFUm,5lAnYvAIkSDNXqfo7DyFUm
4,5,Beggin',Måneskin,980955,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e,3Wrjm47oTz2sjIgck11l5e


In [7]:
len("https://open.spotify.com/track/")

31

In [8]:
test = spotify_data_us_clean["URL"].str
print(test[31:])

0      5HCyWlXZPP0y6Gqq8TgA20
1      4ZtFanR9U6ndgddUvNcjcG
2      3DarAbFujv6eYNliUTyqtz
3      5lAnYvAIkSDNXqfo7DyFUm
4      3Wrjm47oTz2sjIgck11l5e
                ...          
195    2JvzF1RMd7lE3KmFlsyZD8
196    57RA3JGafJm5zRtKJiKPIm
197    7d8dqy6GbiUqotcTSs0jXg
198    3ee8Jmje8o58CHK66QrVC2
199    0u2P5u6lvoDfwTYjAADbn4
Name: URL, Length: 200, dtype: object


In [9]:
token = "BQDkUVHSHlnDbARSqmu8syVDQGhqAeZee8NYPICTtocLBkqD3N-h6ziZHUQVFVyi3N6wtI1lKgmlkwKB2nqq0GTkkDTxNkg1GdyrrUkCbe1g9x4UyaCT2vILcmGU5VzIS9QFcMHqFPJ0aqAyNKb1zcRFwwYgzcoUoLdvIdF1xLjQUYcub8Ey7UrahpmoWr5C7zhnkqV0m6Z4T7dmFoIARROaB8AzsxJD2tJHVef24fMmmice1OxfzHedCoi_KaTkTlT9-E5VZHfOKwGuR76T"
headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.SpotifyTopChartsdb
collection = db.TopUS
collection.drop()

rows=len(spotify_data_us_clean)
global_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    # Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    artist_id = spotify_data_us_clean.iloc[row,5]
    artist_name = spotify_data_us_clean.iloc[row,2]
    track_name = spotify_data_us_clean.iloc[row,1]

    
    response = requests.get(f'https://api.spotify.com/v1/audio-features?ids={artist_id}', headers=headers_dict).json()
    # print(response)
    # artist_id = response["audio_features"]
    # print(response["audio_features"])

    danceability = response["audio_features"][0]["danceability"]
    energy = response["audio_features"][0]["energy"]
    key = response["audio_features"][0]["liveness"]
    loudness = response["audio_features"][0]["key"]
    acousticness = response["audio_features"][0]["acousticness"]
    instrumentalness = response["audio_features"][0]["instrumentalness"]
    liveness = response["audio_features"][0]["liveness"]
    valence = response["audio_features"][0]["valence"]
    tempo = response["audio_features"][0]["tempo"]
    track_href = response["audio_features"][0]["track_href"]
        
        
    global_dict={'Artist Name':artist_name,
                 'Track Name':track_name,
                 'Danceability': danceability,
                 'Energy': energy,
                 'Key': key,
                 'Loudness': loudness,
                 'Acousticness': acousticness,
                 'Instrumentalness': instrumentalness,
                 'Liveness': liveness,
                 'Valence': valence,
                 'Tempo': tempo,
                 'Track Reference': track_href
                 }
    collection.insert_one(global_dict)
#     spotify_artist_dict.append(artist_dict)
   
    print(global_dict)

KeyError: 'audio_features'